#Pip installs, imports, data extraction

In [ ]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 6.2 MB/s 


In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q kaggle

In [ ]:
# download kaggle API credentials from drive
!gdown --id 1m8vWr3w50vqEY88DC8cVKPuMMq1z791E

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1m8vWr3w50vqEY88DC8cVKPuMMq1z791E
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 140kB/s]


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

 33% 5.00M/15.1M [00:00<00:00, 48.1MB/s]
100% 15.1M/15.1M [00:00<00:00, 88.0MB/s]


In [ ]:
!unzip /content/competitive-data-science-predict-future-sales.zip -d /content/kaggle_data

Archive:  /content/competitive-data-science-predict-future-sales.zip
  inflating: /content/kaggle_data/item_categories.csv  
  inflating: /content/kaggle_data/items.csv  
  inflating: /content/kaggle_data/sales_train.csv  
  inflating: /content/kaggle_data/sample_submission.csv  
  inflating: /content/kaggle_data/shops.csv  
  inflating: /content/kaggle_data/test.csv  


In [ ]:
import gc
import pickle
from xgboost import XGBRegressor, DMatrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from matplotlib.pylab import rcParams
import pprint
from functools import partial
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
from itertools import product
import time
import re
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras import callbacks
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, TimeDistributed, Flatten
import keras
from tensorflow.keras.callbacks import EarlyStopping
import pdb

rcParams['figure.figsize'] = 12, 4

#Preprocessing and feature engineering

In [ ]:
def name_correction(x):
    x = x.lower() # all letters lower case
    x = x.partition('[')[0] # partition by square brackets
    x = x.partition('(')[0] # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # remove special characters
    x = x.replace('  ', ' ') # replace double spaces with single spaces
    x = x.strip() # remove leading and trailing white space
    return x

train_path = '/content/kaggle_data/sales_train.csv'
test_path = '/content/kaggle_data/test.csv'
items_path = '/content/kaggle_data/items.csv'
shop_path = '/content/kaggle_data/shops.csv'
categories_path = '/content/kaggle_data/item_categories.csv'
submission_path = '/content/kaggle_data/sample_submission.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
items = pd.read_csv(items_path)
shops = pd.read_csv(shop_path)
cats = pd.read_csv(categories_path)
submission = pd.read_csv(submission_path)

#Outliers Removing

'''
* Removes big outliers
* Remove rows where price is negative
* Makes Values < 1 = 0
'''
train = train[(train.item_price < 300000 )& (train.item_cnt_day < 1000)]
train = train[train.item_price > 0].reset_index(drop = True)
train.loc[train.item_cnt_day < 1, "item_cnt_day"] = 0

#Edit shop duplicates

#Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
#Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
#Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

#Add city and category to shop
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
shops["category"] = shops.shop_name.str.split(" ").map( lambda x: x[1] )
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"

#first
#pdb.set_trace() #type in " continue " to escape and continue running the code


#Only keep shop category >= 5 shops otherwise = other
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )

#Label Encode shops
from sklearn.preprocessing import LabelEncoder
shops["shop_category"] = LabelEncoder().fit_transform(shops.category)
shops["shop_city"] = LabelEncoder().fit_transform(shops.city)
shops = shops[["shop_id", "shop_category", "shop_city"]]

#second
#pdb.set_trace() #type in " continue " to escape and continue running the code

#Cleaning Categorical Data
cats["type_code"] = cats.item_category_name.apply( lambda x: x.split(" ")[0] ).astype(str)
cats.loc[ (cats.type_code == "Игровые")| (cats.type_code == "Аксессуары"), "category" ] = "Игры"

#Only keep categories >= 5 shops otherwise = other
category = []
for cat in cats.type_code.unique():
    if len(cats[cats.type_code == cat]) >= 5: 
        category.append( cat )
cats.type_code = cats.type_code.apply(lambda x: x if (x in category) else "etc")

#Label Encode categories
cats.type_code = LabelEncoder().fit_transform(cats.type_code)
cats["split"] = cats.item_category_name.apply(lambda x: x.split("-"))
cats["subtype"] = cats.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats["subtype_code"] = LabelEncoder().fit_transform( cats["subtype"] )
cats = cats[["item_category_id", "subtype_code", "type_code"]]

#Split item names by first bracket
items["name1"], items["name2"] = items.item_name.str.split("[", 1).str
items["name1"], items["name3"] = items.item_name.str.split("(", 1).str

#Replace special characters and turn to lower case
items["name2"] = items.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items["name3"] = items.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

#Fill nulls with '0'
items = items.fillna('0')

items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))

#Return all characters except the last if name 2 is not "0" - the closing bracket
items.name2 = items.name2.apply( lambda x: x[:-1] if x !="0" else "0")

#Clean item type
items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0])
items.loc[(items.type == "x360") | (items.type == "xbox360") | (items.type == "xbox 360") ,"type"] = "xbox 360"
items.loc[ items.type == "", "type"] = "mac"
items.type = items.type.apply(lambda x: x.replace(" ", "") )
items.loc[(items.type == 'pc' ) | (items.type == 'pс') | (items.type == "pc"), "type" ] = "pc"
items.loc[items.type == 'рs3' , "type"] = "ps3"

#Group by type and count items, replace with others items with count less than 40
group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] < 40:
        drop_cols.append(cat)
items.name2 = items.name2.apply(lambda x: "other" if (x in drop_cols) else x)
items = items.drop(["type"], axis = 1)

#Label Encode items names
items.name2 = LabelEncoder().fit_transform(items.name2)
items.name3 = LabelEncoder().fit_transform(items.name3)

items.drop(["item_name", "name1"],axis = 1, inplace= True)

'''
Create a matrix with every combination of month, shop, and item to increase months.
Item_cnt_day is then summed into an item_cnt_month
'''
ts = time.time()
print('Starting matrix construction...')
matrix = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales = train[train.date_block_num == i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype = np.int16) )

matrix = pd.DataFrame(np.vstack(matrix), columns = cols )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values(cols, inplace = True )
print(f'Matrix effectively produced in {time.time()- ts} seconds')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:103: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: FutureWarning: The default value of regex will change from True to False in a future version.


Starting matrix construction...
Matrix effectively produced in 16.72688126564026 seconds


In [ ]:
#Add revenue to train df
train["revenue"] = train["item_cnt_day"] * train["item_price"]

ts = time.time()
group = train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"]
group.reset_index( inplace = True)
matrix = pd.merge( matrix, group, on = cols, how = "left" )
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)
time.time() - ts

#Create a test set for month 34.
test["date_block_num"] = 34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)

ts = time.time()

#Concatenate train and test sets.
matrix = pd.concat([matrix, test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace = True)
time.time() - ts

#Add shop, items and categories data onto matrix df.
ts = time.time()
matrix = pd.merge( matrix, shops, on = ["shop_id"], how = "left" )
matrix = pd.merge( matrix, items, on = ["item_id"], how = "left" )
matrix = pd.merge( matrix, cats, on = ["item_category_id"], how = "left" )
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["subtype_code"] = matrix["subtype_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)
time.time() - ts

# Define a lag feature function
def lag_feature( df,lags, cols ):
  for col in cols:
      print(col)
      tmp = df[["date_block_num", "shop_id","item_id",col ]]
      for i in lags:
          shifted = tmp.copy()
          shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
          shifted.date_block_num = shifted.date_block_num + i
          df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
  return df

#Add item_cnt_month lag features.
ts = time.time()
matrix = lag_feature( matrix, [1,2,3], ["item_cnt_month"] )
time.time() - ts

#Add the previous month's average item_cnt.
ts = time.time()
group = matrix.groupby( ["date_block_num"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace = True)

#Add lag values of item_cnt_month for month / item_id.
matrix = pd.merge(matrix, group, on = ["date_block_num"], how = "left")
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1], ["date_avg_item_cnt"] )
matrix.drop( ["date_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts


#Add lag values for item_cnt_month for every month / shop combination.
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix.date_item_avg_item_cnt = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3], ['date_item_avg_item_cnt'])
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

#Add lag values for item_cnt_month for month/shop/item.
ts = time.time()
group = matrix.groupby( ["date_block_num","shop_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_avg_item_cnt"] )
matrix.drop( ["date_shop_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts

ts = time.time()
group = matrix.groupby( ["date_block_num","shop_id","item_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_item_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id","item_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_item_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_item_avg_item_cnt"] )
matrix.drop( ["date_shop_item_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts


ts = time.time()
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

#Add lag values for item_cnt_month for month/shop/item subtype.
matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix.date_shop_subtype_avg_item_cnt = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_shop_subtype_avg_item_cnt'])
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts


#Add lag values for item_cnt_month for month/city.
ts = time.time()
group = matrix.groupby(['date_block_num', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_city_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', "shop_city"], how='left')
matrix.date_city_avg_item_cnt = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_city_avg_item_cnt'])
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts


#Add lag values for item_cnt_month for month/city/item.
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'shop_city'], how='left')
matrix.date_item_city_avg_item_cnt = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_item_city_avg_item_cnt'])
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

#Add average item price on to matrix df.
#Add lag values of item price per month.
#Add delta price values - how current month average pirce relates to global average.

ts = time.time()
group = train.groupby( ["item_id"] ).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge( group, on = ["item_id"], how = "left" )
matrix["item_avg_item_price"] = matrix.item_avg_item_price.astype(np.float16)

group = train.groupby( ["date_block_num","item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge(group, on = ["date_block_num","item_id"], how = "left")
matrix["date_item_avg_item_price"] = matrix.date_item_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
matrix = lag_feature( matrix, lags, ["date_item_avg_item_price"] )
for i in lags:
  matrix["delta_price_lag_" + str(i) ] = (matrix["date_item_avg_item_price_lag_" + str(i)]- matrix["item_avg_item_price"] )/ matrix["item_avg_item_price"]

def select_trends(row) :
  for i in lags:
      if row["delta_price_lag_" + str(i)]:
          return row["delta_price_lag_" + str(i)]
  return 0

matrix["delta_price_lag"] = matrix.apply(select_trends, axis = 1)
matrix["delta_price_lag"] = matrix.delta_price_lag.astype( np.float16 )
matrix["delta_price_lag"].fillna( 0 ,inplace = True)

features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]
for i in lags:
  features_to_drop.append("date_item_avg_item_price_lag_" + str(i) )
  features_to_drop.append("delta_price_lag_" + str(i) )

matrix.drop(features_to_drop, axis = 1, inplace = True)
time.time() - ts

#Add total shop revenue per month to matrix df.
#Add lag values of revenue per month.
#Add delta revenue values - how current month revenue relates to global average.

ts = time.time()
group = train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["date_shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(["shop_id"]).agg({ "date_block_num":["mean"] })
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace = True )

matrix = matrix.merge( group, on = ["shop_id"], how = "left" )
matrix["shop_avg_revenue"] = matrix.shop_avg_revenue.astype(np.float32)
matrix["delta_revenue"] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix["delta_revenue"] = matrix["delta_revenue"]. astype(np.float32)

matrix = lag_feature(matrix, [1], ["delta_revenue"])
matrix["delta_revenue_lag_1"] = matrix["delta_revenue_lag_1"].astype(np.float32)
matrix.drop( ["date_shop_revenue", "shop_avg_revenue", "delta_revenue"] ,axis = 1, inplace = True)
time.time() - ts

#Add month and number of days in each month to matrix df.
matrix["month"] = matrix["date_block_num"] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix["days"] = matrix["month"].map(days).astype(np.int8)

ts = time.time()
matrix["item_shop_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')
matrix["item_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id"])["date_block_num"].transform('min')
time.time() - ts

ts = time.time()
matrix = matrix[matrix["date_block_num"] > 3]
time.time() - ts

item_cnt_month
date_avg_item_cnt
date_item_avg_item_cnt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


date_shop_avg_item_cnt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


date_shop_item_avg_item_cnt
date_shop_subtype_avg_item_cnt
date_city_avg_item_cnt
date_item_city_avg_item_cnt
date_item_avg_item_price
delta_revenue


1.5347578525543213

In [ ]:
#need to cast it again as 32 to perform ffill()
matrix = matrix.astype(np.float32)

In [ ]:
matrix.ffill(axis = 0, inplace = True)

In [ ]:
#matrix.ffill().add(matrix.bfill()).div(2)

In [ ]:
data = matrix.copy()
del matrix

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

# MLP approach

In [ ]:
def earlystop(monitor = 'loss', mode = 'min', patience = 5, restore_best_weights = True, verbose = 1):
  es = EarlyStopping(monitor = monitor, mode = mode, patience = patience,restore_best_weights = restore_best_weights, verbose=verbose)
  return es
  
epochs = 10
batch = 64
lr = 0.001
adam = Adam(lr)
es = earlystop()

In [ ]:
model_mlp = Sequential()
model_mlp.add(Dense(50, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)

In [ ]:
model_mlp.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 50)                1650      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,701
Trainable params: 1,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mlp_history = model_mlp.fit(X_train, Y_train, validation_data = (X_valid, Y_valid), epochs=epochs, callbacks=[es], verbose=1)

Epoch 1/10
286818/286818 [==============================] - 730s 3ms/step - loss: 3884.8613 - val_loss: 2.3832
Epoch 2/10
286818/286818 [==============================] - 715s 2ms/step - loss: 30.3831 - val_loss: 1.2930
Epoch 3/10
286818/286818 [==============================] - 717s 3ms/step - loss: 1.4801 - val_loss: 1.2933
Epoch 4/10
286818/286818 [==============================] - 717s 3ms/step - loss: 1.4801 - val_loss: 1.2945
Epoch 5/10
286818/286818 [==============================] - 719s 3ms/step - loss: 1.4801 - val_loss: 1.2957
Epoch 6/10
286818/286818 [==============================] - 719s 3ms/step - loss: 1.4801 - val_loss: 1.2946
Epoch 7/10
286818/286818 [==============================] - 716s 2ms/step - loss: 1.4801 - val_loss: 1.2947
Epoch 8/10
286818/286818 [==============================] - 727s 3ms/step - loss: 1.4801 - val_loss: 1.2945
Epoch 9/10
286818/286818 [==============================] - 733s 3ms/step - loss: 1.4801 - val_loss: 1.2938
Epoch 10/10
286818/28681

In [ ]:
df_his = pd.DataFrame(mlp_history.history)

In [ ]:
predicted_test_data = model_mlp.predict(X_test)

In [ ]:
submission['item_cnt_month'] = predicted_test_data.clip(0,20).round(0)
submission.to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "mlp_features_engineered"

100% 2.14M/2.14M [00:01<00:00, 1.41MB/s]
Successfully submitted to Predict Future Sales

# CNN + LSTM approach

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  keras.layers.LSTM(32, return_sequences=True, input_shape = (X_train.shape[1], 1), dropout = 0.2),
  keras.layers.LSTM(32, return_sequences=True, input_shape = (X_train.shape[1], 1), dropout = 0.2),
  keras.layers.LSTM(32, return_sequences=False, dropout = 0.3),
  keras.layers.Dense(35),
  keras.layers.Dense(10),
  keras.layers.Dense(1),
  #keras.layers.Lambda(lambda x: x * 200)
])
optimizer = Adam(learning_rate=0.001)
model.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics = ['mse'])

In [ ]:
def earlystop(monitor = 'loss', mode = 'min', patience = 5, restore_best_weights = True, verbose = 1):
  es = EarlyStopping(monitor = monitor, mode = mode, patience = patience,restore_best_weights = restore_best_weights, verbose=verbose)
  return es
  
es = earlystop()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 32)          128       
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, None, 32)          8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 35)                1155      
                                                                 
 dense_1 (Dense)             (None, 10)                360       
                                                                 
 dense_2 (Dense)             (None, 1)                 1

In [ ]:
X_valid

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), epochs = 50, batch_size = 1024, verbose = 1, callbacks = [es])

Epoch 1/50
6275/6275 [==============================] - 155s 25ms/step - loss: 8.6662e-04 - mse: 0.0023 - val_loss: 0.0123 - val_mse: 0.0431
Epoch 2/50
6275/6275 [==============================] - 158s 25ms/step - loss: 7.9433e-04 - mse: 0.0023 - val_loss: 0.0120 - val_mse: 0.0366
Epoch 3/50
6275/6275 [==============================] - 156s 25ms/step - loss: 5.0009e-04 - mse: 0.0011 - val_loss: 0.0093 - val_mse: 0.0285
Epoch 4/50
6275/6275 [==============================] - 159s 25ms/step - loss: 4.7705e-04 - mse: 0.0011 - val_loss: 0.0121 - val_mse: 0.0376
Epoch 5/50
6275/6275 [==============================] - 154s 25ms/step - loss: 4.1141e-04 - mse: 9.8904e-04 - val_loss: 0.0148 - val_mse: 0.0416
Epoch 6/50
6275/6275 [==============================] - 160s 25ms/step - loss: 3.9778e-04 - mse: 0.0010 - val_loss: 0.0097 - val_mse: 0.0299
Epoch 7/50
6275/6275 [==============================] - 155s 25ms/step - loss: 4.7090e-04 - mse: 0.0012 - val_loss: 0.0123 - val_mse: 0.0372
Epoch 8/5

In [ ]:
df_his = pd.DataFrame(history.history)

In [ ]:
predicted_test_data = model.predict(X_test)

In [ ]:
submission['item_cnt_month'] = predicted_test_data.clip(0,20).round(0)
submission.to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "CNN + LSTM with feature engineering"